In [ ]:
#run requirements before accessing notebook
%pip install -r requirements.txt

In [ ]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# load config.json
config = json.load(open('config.json'))

In [ ]:
# load data from 2022
data_1 = pd.read_csv(config['2022_data'])
data_1.head()

In [ ]:
# load data from 2023
data_2 = pd.read_json(config['2023_data'])
data_2['experience'] = data_2['experience'].explode()
data_2['operating_mode'] = data_2['operating_mode'].explode()
data_2['type_of_work'] = data_2['type_of_work'].explode()
data_2.head()

In [ ]:
#1. extract salaries horizontally across with link for each employment type option
salaries_df = data_2[['link','salaries']]

# Explode the 'salaries' column
salaries_df = salaries_df.explode('salaries').reset_index(drop=True)

# Normalize the exploded column to extract keys into separate columns
salaries_normalized = pd.json_normalize(salaries_df['salaries'])

# Strip leading and trailing whitespaces in 'employment_type'
salaries_normalized['employment_type'] = salaries_normalized['employment_type'].str.strip()

# Merge normalized salaries with the exploded DataFrame
merged_df = pd.concat([salaries_df.drop('salaries', axis=1), salaries_normalized], axis=1)
merged_df['lower'] = merged_df['lower'].apply(lambda x: x.replace(' ', '') if x != None else x).astype(float)
merged_df['upper'] = merged_df['upper'].apply(lambda x: x.replace(' ', '') if x != None else x).astype(float)

# Pivot the DataFrame to create separate columns for each employment type
pivot_df = merged_df.pivot_table(
    index='link',
    columns='employment_type',
    values=['lower', 'upper'],
    aggfunc='first'
)

# Flatten the multi-level column index
pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]

# Reset index to bring 'link' back as a column
pivot_df = pivot_df.reset_index()

# Merge with the original 'link' column
salaries_df = salaries_df[['link']].merge(pivot_df, on='link', how='left').drop_duplicates()

salaries_df.head()

In [ ]:
#2. extract skills to 12 columns, replacing values for "regular" etc
skills_df = data_2[['link','skills']]

# Mapping values to new values
value_mapping = {
    "master": 5,
    "advanced": 4,
    "regular": 3,
    "junior": 2,
    "nice to have": 1
}

# Replace values in the 'skills' column based on the mapping
skills_df.loc[:, 'skills'] = skills_df['skills'].apply(lambda x: {k: value_mapping.get(v, v) for k, v in x.items()})

# Create a function to extract keys and values
def extract_skills(row):
    skills_dict = row['skills']
    skills_keys = list(skills_dict.keys())
    skills_values = list(skills_dict.values())
    return pd.Series(skills_keys + skills_values, index=['skills_name_' + str(i) for i in range(len(skills_keys))] +
                                                     ['skills_value_' + str(i) for i in range(len(skills_values))])

# Apply the function across the DataFrame
skills_extracted = skills_df.apply(extract_skills, axis=1)

# Concatenate the extracted skills with the 'link' column
skills_df = pd.concat([skills_df['link'], skills_extracted], axis=1)

# # Pivot the DataFrame
# skills_result_df = skills_result_df.pivot_table(
#     index='link',
#     aggfunc='first'
# )

# Convert specific columns to integer type
int_columns = [col for col in skills_df.columns if 'skills_value_' in col]
skills_df[int_columns] = skills_df[int_columns].astype(pd.Int64Dtype())

skills_df.head()

In [ ]:
#3. extract a subset of data from data_1 for a last month
data_1['Published_at'] = pd.to_datetime(data_1['Published_at'])

# Find the maximum date
max_date = data_1['Published_at'].max()

# Subtract one month from the maximum date
start_date = max_date - pd.DateOffset(days=45) #the date is arbitrary based on the estimations between initial commit and minimal date

# Filter the DataFrame based on the condition
data_1 = data_1[data_1['Published_at'] >= start_date]

# Filter rows containing 'data' and 'engineer' in 'Title' column (case insensitive)
data_1 = data_1[data_1['Title'].str.contains('data', case=False) & data_1['Title'].str.contains('engineer', case=False)]

data_1.head()

In [ ]:
#4. create final data_2 object
#merge data_2
data_2 = pd.merge(data_2, salaries_df, on='link', how='inner')
data_2 = pd.merge(data_2, skills_df, on='link', how='inner')

#drop columns
data_2 = data_2.drop(['salaries', 'skills'], axis=1)

# Filter rows containing 'data' and 'engineer' in 'name' column (case insensitive)
data_2 = data_2[data_2['name'].str.contains('data', case=False) & data_2['name'].str.contains('engineer', case=False)]

data_2.head()

In [ ]:
#5. create a data model for a target data frame and append rows there
df = pd.DataFrame(
    {
        'title'            : pd.Series(dtype=str),
        'category'         : pd.Series(dtype=str),
        'location'         : pd.Series(dtype=str),
        'experience'       : pd.Series(dtype=str),
        'operating_mode'   : pd.Series(dtype=str),
        'salary_lower_B2B' : pd.Series(dtype=float),
        'salary_upper_B2B' : pd.Series(dtype=float),
        'salary_lower_UoP' : pd.Series(dtype=float),
        'salary_upper_UoP' : pd.Series(dtype=float),
        'skills_name_0'    : pd.Series(dtype=str),
        'skills_value_0'   : pd.Series(dtype=int),
        'skills_name_1'    : pd.Series(dtype=str),
        'skills_value_1'   : pd.Series(dtype=int),
        'skills_name_2'    : pd.Series(dtype=str),
        'skills_value_2'   : pd.Series(dtype=int),
        'year'             : pd.Series(dtype=int)
    },
    index=pd.Index(range(0), name='id'))

data_1 = data_1[['Title','Marker_icon','City','Experience_level','Workplace_type','salary_from_b2b','salary_to_b2b',
                 'salary_from_permanent', 'salary_to_permanent', 'skills_name_0', 'skills_value_0', 'skills_name_1',
                 'skills_value_1', 'skills_name_2', 'skills_value_2']]
data_1['year'] = 2022
data_1.columns = df.columns
data_1.replace(0, np.nan, inplace=True)

df = df._append(data_1)

data_2 = data_2[['name','category','location','experience','operating_mode','lower_B2B','upper_B2B',
                 'lower_Permanent', 'upper_Permanent', 'skills_name_0', 'skills_value_0', 'skills_name_1',
                 'skills_value_1', 'skills_name_2', 'skills_value_2']]
data_2['year'] = 2023
data_2.columns = df.columns

df = df._append(data_2)

df.head()

In [ ]:
#get info about dataframe
df.info()